In [3]:
import random
from datetime import datetime, timedelta
import pymysql
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# MySQL 연결
connection = pymysql.connect(
    host='database-1.cbcw28i2we7h.us-east-2.rds.amazonaws.com',
    user='kj',
    password='1234',
    database='nahonlab',
    charset='utf8mb4'
)
cursor = connection.cursor()

# 데이터 생성 설정
num_users = 30  # user_id 3부터 32까지
start_date = datetime(2024, 11, 1)
end_date = datetime(2024, 11, 30)

# 사용자별 초기 값 설정
initial_metrics = {
    user_id: {
        "height": random.randint(160, 190),  # 고정된 키 값
        "weight": random.uniform(50, 80),
        "fat_percent": random.uniform(15, 30),
        "muscle_mass": random.uniform(25, 40),
    }
    for user_id in range(3, 33)  # user_id 3부터 32까지
}

# 기본 값 생성 함수
def generate_metrics(user_id, prev_weight, prev_fat_percent, prev_muscle_mass):
    calories_intake = random.randint(1800, 3500)  # 섭취 칼로리
    calories_burned = random.randint(500, 1000)   # 소모 칼로리

    # 체중 변화: 섭취 > 소모 -> 증가, 섭취 < 소모 -> 감소
    calorie_diff = calories_intake - calories_burned
    weight_change = calorie_diff / 7000  # 1kg = 약 7000kcal
    new_weight = max(45, prev_weight + weight_change)  # 체중은 최소 45kg

    # 체지방률 변화: 섭취 > 소모 -> 증가, 운동량 높으면 감소
    fat_change = 0.2 if calorie_diff > 500 else (-0.1 if calories_burned > 800 else 0)
    new_fat_percent = max(5, min(40, prev_fat_percent + fat_change))  # 체지방률 5~40% 제한

    # 근육량 변화: 소모 칼로리가 높으면 증가, 섭취만 높으면 유지
    muscle_change = 0.1 if calories_burned > 700 else (-0.05 if calorie_diff > 0 else 0.02)
    new_muscle_mass = max(20, prev_muscle_mass + muscle_change)  # 근육량 최소 20kg

    return new_weight, new_fat_percent, new_muscle_mass, calories_burned, calories_intake

# 데이터 생성 및 삽입
for user_id in range(3, 33):
    user_metrics = initial_metrics[user_id]
    prev_weight = user_metrics["weight"]
    prev_fat_percent = user_metrics["fat_percent"]
    prev_muscle_mass = user_metrics["muscle_mass"]
    height = user_metrics["height"]  # 고정된 키 값

    date = start_date
    while date <= end_date:
        # 새로운 데이터 생성
        new_weight, new_fat_percent, new_muscle_mass, calories_burned, calories_intake = generate_metrics(
            user_id, prev_weight, prev_fat_percent, prev_muscle_mass
        )

        # 데이터 삽입
        cursor.execute(
            """
            INSERT INTO user_body_metrics (user_id, record_date, height, weight, fat_percent, muscle_mass, calories_burned, calories_intake)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """,
            (
                user_id,
                date.strftime('%Y-%m-%d'),
                height,  # 고정된 키 값 사용
                round(new_weight, 2),
                round(new_fat_percent, 2),
                round(new_muscle_mass, 2),
                calories_burned,
                calories_intake
            )
        )

        # 이전 값을 업데이트
        prev_weight = new_weight
        prev_fat_percent = new_fat_percent
        prev_muscle_mass = new_muscle_mass

        # 다음 날짜로 이동
        date += timedelta(days=1)

# 데이터베이스 커밋
connection.commit()

# 데이터 가져오기 및 상관관계 분석
query = """
SELECT weight, record_date, fat_percent, muscle_mass, calories_burned, calories_intake
FROM user_body_metrics
"""
data = pd.read_sql(query, connection)

# 상관관계 계산 및 시각화
correlation_matrix = data.corr()
print("Correlation Matrix:")
print(correlation_matrix)

sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm")
plt.title("Correlation Matrix of User Body Metrics")
plt.show()

# 연결 닫기
cursor.close()
connection.close()

print("Data generation, insertion, and correlation analysis complete.")


In [7]:
import pandas as pd
import numpy as np
import pymysql
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import joblib

# MySQL 데이터베이스 연결 설정
connection = pymysql.connect(
    host='database-1.cbcw28i2we7h.us-east-2.rds.amazonaws.com',
    user='kj',
    password='1234',
    database='nahonlab',
    charset='utf8mb4'
)

# 데이터 불러오기
query = """
SELECT user_id, record_date, weight, fat_percent, muscle_mass, calories_burned, calories_intake
FROM user_body_metrics
"""
data = pd.read_sql(query, connection)
connection.close()

# 날짜 형식 변환 및 정렬
data['record_date'] = pd.to_datetime(data['record_date'])
data = data.sort_values(['user_id', 'record_date'])

# 피처와 타깃 변수 정의
features = data[['calories_burned', 'calories_intake']]
targets = data[['weight', 'fat_percent', 'muscle_mass']]

# 정규화
scaler_features = MinMaxScaler()
scaler_targets = MinMaxScaler()

if not features.empty:
    features_scaled = scaler_features.fit_transform(features)
    targets_scaled = scaler_targets.fit_transform(targets)
else:
    raise ValueError("데이터셋이 비어있습니다. 학습할 데이터가 필요합니다.")

# 시계열 데이터 생성 함수
def create_sequences(features, targets, time_steps=15):
    X, y = [], []
    for i in range(len(features) - time_steps):
        X.append(features[i:(i + time_steps)])
        y.append(targets[i + time_steps])
    return np.array(X), np.array(y)

# 시퀀스 데이터 생성
TIME_STEPS = 15
X, y = create_sequences(features_scaled, targets_scaled, TIME_STEPS)

# 학습 및 테스트 데이터 분리
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# 단일 모델 생성 및 학습
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, return_sequences=False, input_shape=input_shape))
    model.add(Dense(3))  # 예측할 타깃 변수의 수
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

model = create_lstm_model((X_train.shape[1], X_train.shape[2]))
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, shuffle=False)

# 모델 저장
model.save("global_model.h5")
joblib.dump(scaler_features, "global_scaler_features.pkl")
joblib.dump(scaler_targets, "global_scaler_targets.pkl")

print("전체 데이터를 사용하여 모델 학습 완료.")


/tmp/ipykernel_111397/1913586348.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, connection)


Epoch 1/50


2024-12-11 21:13:17.549467: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-12-11 21:13:17.946422: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fb7540a80a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-12-11 21:13:17.946561: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2060, Compute Capability 7.5
2024-12-11 21:13:17.953191: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1733919198.109456  111482 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


36/36 [==============================] - 3s 14ms/step - loss: 0.1285 - val_loss: 0.1292
Epoch 2/50
36/36 [==============================] - 0s 4ms/step - loss: 0.0875 - val_loss: 0.0936
Epoch 3/50
36/36 [==============================] - 0s 7ms/step - loss: 0.0847 - val_loss: 0.0808
Epoch 4/50
36/36 [==============================] - 0s 7ms/step - loss: 0.0788 - val_loss: 0.0850
Epoch 5/50
36/36 [==============================] - 0s 4ms/step - loss: 0.0776 - val_loss: 0.0843
Epoch 6/50
36/36 [==============================] - 0s 4ms/step - loss: 0.0758 - val_loss: 0.0846
Epoch 7/50
36/36 [==============================] - 0s 7ms/step - loss: 0.0744 - val_loss: 0.0848
Epoch 8/50
36/36 [==============================] - 0s 5ms/step - loss: 0.0732 - val_loss: 0.0853
Epoch 9/50
36/36 [==============================] - 0s 8ms/step - loss: 0.0721 - val_loss: 0.0858
Epoch 10/50
36/36 [==============================] - 0s 7ms/step - loss: 0.0712 - val_loss: 0.0863
Epoch 11/50
36/36 [==========

/home/hyun/venv/mldl_venv/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [24]:
import joblib
import numpy as np
import tensorflow.keras.models as keras_models
import pandas as pd
import pymysql

# 시계열 데이터의 길이 (시간 단계)
TIME_STEPS = 15

# 예측 함수
def predict_for_user(user_id, predict_days=15):
    # MySQL 데이터베이스 연결 설정
    connection = pymysql.connect(
        host='database-1.cbcw28i2we7h.us-east-2.rds.amazonaws.com',
        user='kj',
        password='1234',
        database='nahonlab',
        charset='utf8mb4'
    )

    # 저장된 모델 및 스케일러 로드
    try:
        model = keras_models.load_model("global_model.h5")
        scaler_features = joblib.load("global_scaler_features.pkl")
        scaler_targets = joblib.load("global_scaler_targets.pkl")
    except FileNotFoundError:
        print("저장된 글로벌 모델 또는 스케일러가 없습니다.")
        connection.close()  # 연결 종료
        return

    # 데이터 불러오기
    try:
        data = pd.read_sql("""
        SELECT user_id, record_date, weight, fat_percent, muscle_mass, calories_burned, calories_intake
        FROM user_body_metrics
        """, connection)
    except Exception as e:
        print(f"데이터베이스 쿼리 실행 중 오류 발생: {e}")
        connection.close()  # 연결 종료
        return

    user_data = data[data['user_id'] == user_id].copy()
    features = user_data[['calories_burned', 'calories_intake']]

    if features.empty:
        print(f"User {user_id}에 대한 데이터가 없습니다.")
        connection.close()  # 연결 종료
        return

    features_scaled = scaler_features.transform(features)

    if len(features_scaled) < TIME_STEPS:
        print(f"User {user_id}에 대한 충분한 데이터가 없어 예측을 수행할 수 없습니다.")
        connection.close()  # 연결 종료
        return

    last_15_days_features = features_scaled[-TIME_STEPS:]
    current_sequence = last_15_days_features
    predictions_scaled = []

    for _ in range(predict_days):
        next_pred_scaled = model.predict(current_sequence[np.newaxis, :, :])[0]
        predictions_scaled.append(next_pred_scaled)
        next_feature = np.array([next_pred_scaled[0], next_pred_scaled[1]])
        current_sequence = np.vstack((current_sequence[1:], next_feature))

    predictions = scaler_targets.inverse_transform(predictions_scaled)

    # 연결 종료
    connection.close()
    return predictions

# 예측 결과 출력 함수
def display_predictions(user_id, predict_days=15):
    predictions = predict_for_user(user_id, predict_days)
    if predictions is not None:
        print(f"User {user_id}의 다음 {predict_days}일 예측 결과:")
        for day, pred in enumerate(predictions, 1):
            print(f"Day {day}: 체중={pred[0]:.2f}kg, 체지방률={pred[1]:.2f}%, 근육량={pred[2]:.2f}kg")

# 예시로 특정 사용자 예측 (예: 10일 예측)
display_predictions(user_id=5, predict_days=15)


2024-12-12 16:42:22.207867: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-12 16:42:22.207915: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-12 16:42:22.208885: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-12 16:42:22.214631: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-12 16:42:22.923106: W tensorflow/compiler/tf2

1/1 [==============================] - 0s 13ms/step


2024-12-12 16:42:25.977312: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


1/1 [==============================] - 0s 15ms/step
User 5의 다음 15일 예측 결과:
Day 1: 체중=70.54kg, 체지방률=26.07%, 근육량=34.85kg
Day 2: 체중=69.78kg, 체지방률=26.03%, 근육량=34.75kg
Day 3: 체중=69.50kg, 체지방률=26.08%, 근육량=34.77kg
Day 4: 체중=69.52kg, 체지방률=26.10%, 근육량=34.80kg
Day 5: 체중=69.72kg, 체지방률=26.09%, 근육량=34.83kg
Day 6: 체중=70.34kg, 체지방률=26.01%, 근육량=34.85kg
Day 7: 체중=70.63kg, 체지방률=25.97%, 근육량=34.86kg
Day 8: 체중=72.03kg, 체지방률=25.77%, 근육량=34.84kg
Day 9: 체중=71.52kg, 체지방률=25.82%, 근육량=34.87kg
Day 10: 체중=71.04kg, 체지방률=25.90%, 근육량=34.91kg
Day 11: 체중=71.31kg, 체지방률=25.89%, 근육량=34.93kg
Day 12: 체중=71.93kg, 체지방률=25.81%, 근육량=34.92kg
Day 13: 체중=71.67kg, 체지방률=25.85%, 근육량=34.94kg
Day 14: 체중=71.86kg, 체지방률=25.84%, 근육량=34.95kg
Day 15: 체중=71.11kg, 체지방률=25.95%, 근육량=34.98kg


In [8]:
import pymysql
from datetime import datetime, timedelta
import random

# MySQL 데이터베이스 연결 설정
connection = pymysql.connect(
    host='database-1.cbcw28i2we7h.us-east-2.rds.amazonaws.com',
    user='kj',
    password='1234',
    database='nahonlab',
    charset='utf8mb4'
)

cursor = connection.cursor()

# 최근 기록을 가져오는 쿼리
query_latest_metrics = """
SELECT 
    user_id, 
    weight AS current_weight,
    fat_percent AS current_fat_percent,
    muscle_mass AS current_muscle_mass,
    height / 100.0 AS height_m -- 키를 m 단위로 변환
FROM user_body_metrics AS ubm
WHERE record_date = (
    SELECT MAX(record_date) 
    FROM user_body_metrics 
    WHERE user_id = ubm.user_id
)
AND user_id BETWEEN 3 AND 32;
"""

cursor.execute(query_latest_metrics)
latest_metrics = cursor.fetchall()

# 목표 생성 및 삽입
insert_goals_query = """
INSERT INTO user_goals (user_id, target_weight, target_fat_per, target_muscle_mass, start_date, end_date)
VALUES (%s, %s, %s, %s, %s, %s);
"""

start_date = datetime.now().date()

for row in latest_metrics:
    user_id, current_weight, current_fat_percent, current_muscle_mass, height_m = row

    # 현재 BMI 계산
    current_bmi = current_weight / (height_m ** 2)

    # 체중 목표 설정 (BMI 기반 + 편차)
    if current_bmi < 16:
        base_weight_change = 5
    elif current_bmi < 18.5:
        base_weight_change = 3
    elif current_bmi > 30:
        base_weight_change = -5
    elif current_bmi > 25:
        base_weight_change = -3
    else:
        base_weight_change = 0
    
    target_weight = current_weight + base_weight_change + random.uniform(-2, 2)  # 무작위 편차 추가

    # 체지방률 목표 설정 (기본 변화량 + 편차)
    if current_fat_percent > 25:
        base_fat_change = -5
    elif current_fat_percent < 10:
        base_fat_change = 3
    else:
        base_fat_change = 0
    
    target_fat_percent = current_fat_percent + base_fat_change + random.uniform(-3, 3)  # 무작위 편차 추가

    # 근육량 목표 설정 (기본 변화량 + 편차)
    target_muscle_mass = current_muscle_mass + 2 + random.uniform(-0.5, 0.5)  # 무작위 편차 추가

    # 목표 기간 설정: 30일 ~ 90일 랜덤
    goal_duration = random.randint(30, 90)
    end_date = start_date + timedelta(days=goal_duration)

    # 목표 삽입
    cursor.execute(insert_goals_query, (
        user_id,
        round(target_weight, 2),
        round(target_fat_percent, 2),
        round(target_muscle_mass, 2),
        start_date,
        end_date
    ))

# 데이터베이스 커밋 및 연결 종료
connection.commit()
cursor.close()
connection.close()

print("user_goals 테이블에 목표 데이터 삽입 완료.")


user_goals 테이블에 목표 데이터 삽입 완료.


In [38]:
import pymysql
import pandas as pd
from datetime import date

# 데이터베이스 연결 설정
connection = pymysql.connect(
    host='database-1.cbcw28i2we7h.us-east-2.rds.amazonaws.com',
    user='kj',
    password='1234',
    database='nahonlab',
    charset='utf8mb4'
)

def calculate_age(birthday):
    """생년월일로부터 만 나이 계산"""
    today = date.today()
    age = today.year - birthday.year
    
    # 생일이 아직 오지 않았으면 나이 1 감소
    if (today.month, today.day) < (birthday.month, birthday.day):
        age -= 1
    
    return age

def calculate_base_calories(user):
    """기초대사율(BMR) 계산"""
    current_weight = user['current_weight']
    current_height = user['current_height']
    age = calculate_age(user['birthday'])
    sex = user['sex']
    
    if sex == 'male':
        # 헤리스-베네딕트 공식 (남성)
        bmr = 88.362 + (13.397 * current_weight) + (4.799 * current_height) - (5.677 * age)
        activity_multiplier = 1.55  # 중간 정도 활동적
    else:
        # 헤리스-베네딕트 공식 (여성)
        bmr = 447.593 + (9.247 * current_weight) + (3.098 * current_height) - (4.330 * age)
        activity_multiplier = 1.45  # 중간 정도 활동적
    
    tdee = bmr * activity_multiplier
    
    # 로그 출력
    print(f"User ID: {user['user_id']}, BMR: {bmr:.2f}, TDEE: {tdee:.2f}")
    return tdee

def calculate_macro_goals(user, diet_strategy):
    """개인화된 매크로 영양소 목표 설정"""
    base_calories = calculate_base_calories(user)
    
    # 체중 관리 목표에 따른 칼로리 조정
    if 'weight_loss' in diet_strategy:
        base_calories -= 300  # 체중 감량을 위해 칼로리 감소
    elif 'muscle_gain' in diet_strategy:
        base_calories += 300  # 근육 증가를 위해 칼로리 증가
    
    # 매크로 비율 결정
    if 'muscle_gain' in diet_strategy:
        protein_ratio, carb_ratio, fat_ratio = 0.3, 0.4, 0.3
    elif 'weight_loss' in diet_strategy:
        protein_ratio, carb_ratio, fat_ratio = 0.35, 0.3, 0.35
    else:
        protein_ratio, carb_ratio, fat_ratio = 0.3, 0.4, 0.3
    
    # 단백질 목표 계산 (체중 기반)
    base_protein = user['current_weight'] * (
        2.2 if 'muscle_gain' in diet_strategy else 
        1.6 if 'weight_loss' in diet_strategy else 
        1.8
    )
    
    # 탄수화물, 지방 목표 계산
    base_carbs = (base_calories * carb_ratio) / 4
    base_fat = (base_calories * fat_ratio) / 9
    
    # 로그 출력
    calculated_total_calories = (base_protein * 4) + (base_carbs * 4) + (base_fat * 9)
    print(f"User ID: {user['user_id']}  "
          f"Protein: {base_protein:.2f}g, Carbs: {base_carbs:.2f}g, Fat: {base_fat:.2f}g, "
          f"Target Calories: {calculated_total_calories:.2f}")

def determine_diet_strategy(user):
    """개인화된 식단 전략 결정"""
    strategies = []
    
    # 체중 변화 전략
    weight_diff = user['target_weight'] - user['current_weight']
    if weight_diff < -2:  # 체중 감량
        strategies.append('weight_loss')
    elif weight_diff > 2:  # 체중 증가
        strategies.append('muscle_gain')
    else:
        strategies.append('maintenance')
    
    # 나이 전략
    age = calculate_age(user['birthday'])
    if age < 25:
        strategies.append('young_high_metabolism')
    elif age > 40:
        strategies.append('slower_metabolism')
    
    # 성별 및 체성분 전략
    if user['sex'] == 'MALE':
        strategies.append('male_muscle_priority')
        if user['current_fat_percent'] > 20:
            strategies.append('fat_reduction')
    else:
        strategies.append('female_lean_priority')
        if user['current_fat_percent'] > 25:
            strategies.append('fat_reduction')
    
    # 근육량 전략
    if user['current_muscle_mass'] < user['target_muscle_mass']:
        strategies.append('muscle_build')
    
    return strategies

try:
    # 데이터베이스에서 데이터 가져오기
    with connection.cursor() as cursor:
        # 통합 쿼리: 사용자 정보, 신체 측정값, 목표 결합
        query = """
        SELECT 
            ui.user_id,
            ui.birthday,
            ui.sex,
            ubm.weight AS current_weight,
            ubm.height AS current_height,
            ubm.fat_percent AS current_fat_percent,
            ubm.muscle_mass AS current_muscle_mass,
            ug.target_weight,
            ug.target_fat_per,
            ug.target_muscle_mass
        FROM user_info ui
        JOIN user_body_metrics ubm ON ui.user_id = ubm.user_id
        JOIN user_goals ug ON ui.user_id = ug.user_id
        WHERE ubm.record_date = (
            SELECT MAX(record_date) 
            FROM user_body_metrics 
            WHERE user_id = ubm.user_id
        )
        """
        
        cursor.execute(query)
        metrics_data = cursor.fetchall()
        metrics_columns = [col[0] for col in cursor.description]
        metrics_df = pd.DataFrame(metrics_data, columns=metrics_columns)

finally:
    connection.close()

# 사용자 ID 입력받아 정보 출력
input_user_id = int(input("Enter User ID: "))
user_data = metrics_df[metrics_df['user_id'] == input_user_id]

if not user_data.empty:
    user = user_data.iloc[0]
    diet_strategy = determine_diet_strategy(user)
    calculate_macro_goals(user, diet_strategy)
else:
    print(f"User ID {input_user_id} not found.")


User ID: 18, BMR: 1652.98, TDEE: 2396.82
User ID: 18  Protein: 148.00g, Carbs: 239.68g, Fat: 79.89g, Target Calories: 2269.76
